## Using VGG16 for prediction of age and gender of the person. 

In [2]:
import numpy as np
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator

In [3]:
folder_path = 'archive/utkface_aligned_cropped/UTKFace'

In [4]:
# I want to make a dataframe having first column as age, then gender and last image name.
age = []
gender = []
img_path = []
for file in os.listdir(folder_path):
    age.append(int(file.split('_')[0]))
    gender.append(int(file.split('_')[1]))
    img_path.append(file)

print(f"Number of images: {len(gender)}")

Number of images: 23708


In [5]:
df = pd.DataFrame({'age': age, 'gender':gender, 'img_name':img_path})
print(f"Number of rows and columns: {df.shape}")
print(df.head())

Number of rows and columns: (23708, 3)
   age  gender                               img_name
0    9       1   9_1_2_20161219204347420.jpg.chip.jpg
1   36       0  36_0_1_20170117163203851.jpg.chip.jpg
2   86       1  86_1_0_20170120225751953.jpg.chip.jpg
3   26       1  26_1_0_20170116171048641.jpg.chip.jpg
4    1       1   1_1_2_20161219154612988.jpg.chip.jpg


In [6]:
# splitting the data into test and train dataset. 
train_df = df.sample(frac=1, random_state= 0).iloc[:20000] 
test_df = df.sample(frac=1, random_state =0).iloc[20000:]
print(f"Number of rows and columns in training: {train_df.shape}")
print(f"Number of rows and columns in testing: {test_df.shape}")

Number of rows and columns in training: (20000, 3)
Number of rows and columns in testing: (3708, 3)


In [7]:
train_dg = ImageDataGenerator(
    rescale=1./255,               # This rescales the pixel values from the range [0, 255] to [0, 1].
    rotation_range=30,            # Randomly rotates images by up to 30 degrees.
    width_shift_range=0.2,        # Randomly shifts images horizontally by up to 20% of the width.
    height_shift_range=0.2,       # Randomly shifts images vertically by up to 20% of the height.
    shear_range=0.2,              # Applies random shear transformations.
    zoom_range=0.2,               # Randomly zooms into images by up to 20%.
    horizontal_flip=True          # Randomly flips images horizontally.
)

test_dg = ImageDataGenerator(rescale=1./255)

In [8]:
# Generator
train_gen = train_dg.flow_from_dataframe(train_df, 
                                               directory=folder_path,
                                               x_col='img_name', 
                                               y_col=['age', 'gender'],          # Specifies the columns in train_df that contain the target variables.
                                               target_size=(200, 200),           # Resizes all images to 200x200 pixels.
                                               class_mode='multi_output'         # Indicates that this is a multi-output problem.
                                               )

test_gen = test_dg.flow_from_dataframe(test_df, 
                                               directory=folder_path,
                                               x_col='img_name', 
                                               y_col=['age', 'gender'],          # Specifies the columns in train_df that contain the target variables.
                                               target_size=(200, 200),           # Resizes all images to 200x200 pixels.
                                               class_mode='multi_output'         # Indicates that this is a multi-output problem.
                                               )

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [9]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import *

In [20]:
# In VGG16 I don't want to make any changes in the weights of the convolutional layers. 
# vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))
vgg16 = VGG16(include_top=False, input_shape=(200,200,3))

In [21]:
# Don't want to change the weights of the VGG16.
vgg16.trainable = False

# Storing output from the last layer of VGG16 to "output"
outputs = vgg16.layers[-1].output

# Flattening the output.
flatten = Flatten()(outputs)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512, activation='relu')(flatten)

dense11 = Dense(512, activation='relu')(dense1)
dense22 = Dense(512, activation='relu')(dense2)

output1 = Dense(1, activation='linear', name = 'age')(dense11)
output2 = Dense(1, activation='sigmoid', name = 'gender')(dense22)

In [22]:
model = Model(inputs=vgg16.input, outputs=[output1, output2])

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 200, 200, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 200, 200, 64)         36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)         0         ['block1_conv2[0][0]']        
                                                                                            

In [24]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'}, loss_weights={'age': 1, 'gender': 99})

In [25]:
history = model.fit(train_gen, batch_size=32, epochs=10, validation_data=test_gen)

Epoch 1/10
 36/625 [>.............................] - ETA: 34:13 - loss: 112.0647 - age_loss: 16.1003 - gender_loss: 0.9693 - age_mae: 16.1003 - gender_accuracy: 0.5703

KeyboardInterrupt: 